## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2026-01-13-18-54-31 +0000,nyt,Justices Seem Inclined to Allow States to Bar ...,https://www.nytimes.com/2026/01/13/us/politics...
1,2026-01-13-18-53-53 +0000,nypost,Target faces protests after ICE detains US cit...,https://nypost.com/2026/01/13/business/target-...
2,2026-01-13-18-51-22 +0000,bbc,Republicans vow to hold Bill Clinton in contem...,https://www.bbc.com/news/articles/cj6w17ykd89o...
3,2026-01-13-18-50-11 +0000,nyt,Here’s the latest on the argument.,https://www.nytimes.com/2026/01/13/us/politics...
4,2026-01-13-18-50-00 +0000,wsj,Iran Is Hunting Down Starlink Users to Stop Pr...,https://www.wsj.com/world/middle-east/iran-is-...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2379/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
58,trump,70
30,iran,32
487,fed,21
11,ice,17
135,new,15


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
140,2026-01-13-14-47-00 +0000,wsj,Some Trump administration officials and allies...,https://www.wsj.com/politics/policy/trump-alli...,156
343,2026-01-12-22-04-31 +0000,cbc,Bank of Canada governor defends U.S. Fed's Pow...,https://www.cbc.ca/news/politics/bank-of-canad...,149
374,2026-01-12-20-28-53 +0000,nyt,Deposed Shah’s Son Hopes Trump Will Put Iran R...,https://www.nytimes.com/2026/01/11/world/europ...,142
143,2026-01-13-14-35-20 +0000,nyt,Trump Says He Will Impose Tariffs on Iran’s Tr...,https://www.nytimes.com/2026/01/13/world/middl...,132
353,2026-01-12-21-30-49 +0000,nypost,Trump pushed to take ‘immediate action’ in Ira...,https://nypost.com/2026/01/12/us-news/lawmaker...,130


## Select top headlines

In [5]:
with open('exclude_sources.txt') as f:
    excluded_sources = {
        line.strip()
        for line in f
        if line.strip() and not line.strip().startswith('#')
    }

top_count = 10
top_rows = []
working = word_scores.copy()
remaining = latest.copy()

while len(top_rows) < top_count and not remaining.empty:
    scored_remaining = remaining.assign(
        score=remaining['title'].apply(
            lambda t: sum(
                working.get(w.lower(), 0)
                for w in re.findall(r'[A-Za-z]+', t)
                if len(w) > 1
            )
        )
    )
    ranked_loop = scored_remaining.sort_values('score', ascending=False)

    if ranked_loop.empty:
        break

    top_story = ranked_loop.iloc[0]
    remaining = remaining.drop(top_story.name)

    if top_story['source'] in excluded_sources:
        continue

    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for word in words:
        working.pop(word, None)

    top_rows.append(top_story[['score', 'pubdate', 'source', 'title', 'link']])

top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
140,156,2026-01-13-14-47-00 +0000,wsj,Some Trump administration officials and allies...,https://www.wsj.com/politics/policy/trump-alli...
316,74,2026-01-12-23-21-39 +0000,nypost,Iran to hang 26-year-old in alleged first exec...,https://nypost.com/2026/01/12/world-news/iran-...
132,61,2026-01-13-15-06-00 +0000,wsj,The Justice Department’s investigation of Fede...,https://www.wsj.com/economy/central-banking/th...
137,56,2026-01-13-15-03-31 +0000,nypost,"Trump tells Iranians ‘keep protesting,’ says ‘...",https://nypost.com/2026/01/13/us-news/trump-te...
254,51,2026-01-13-04-55-39 +0000,wapo,ICE arrests New York City Council employee at ...,https://www.washingtonpost.com/immigration/202...
315,46,2026-01-12-23-31-24 +0000,nypost,"Hegseth tells defense contractors to ‘step up,...",https://nypost.com/2026/01/12/us-news/hegseth-...
270,43,2026-01-13-03-06-52 +0000,nypost,Hakeem Jeffries calls for Kristi Noem ouster a...,https://nypost.com/2026/01/12/us-news/hakeem-j...
338,41,2026-01-12-22-21-49 +0000,nypost,Supreme Court to dive into transgender sports ...,https://nypost.com/2026/01/12/us-news/supreme-...
377,41,2026-01-12-20-13-22 +0000,nypost,Hundreds of murdered Iranian protesters found ...,https://nypost.com/2026/01/12/world-news/fashi...
177,39,2026-01-13-13-27-00 +0000,wsj,South Korean Prosecutors Seek Death Penalty fo...,https://www.wsj.com/world/asia/south-korean-pr...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
